In [ ]:
import psycopg2
import matplotlib.pyplot as plt
import numpy as np
import math

%matplotlib inline

import pandas as pd
from pandas.plotting import scatter_matrix

# https://pandas.pydata.org/pandas-docs/stable/visualization.html#plotting-tools

In [ ]:
dbName = 'nBody'
dbHost = 'localhost'
dbUsername = 'username'
dbPassword = 'password'

In [ ]:
conn = psycopg2.connect("dbname='{0}' user='{1}' host='{2}' password='{3}'".format(dbName, dbUsername, dbHost, dbPassword))
db = conn.cursor()

In [ ]:
#non-resonant pid => 2930773953
#resonant pid => 87897423
db.execute("select a, e, i, ascendingNode, l, f  from particlesimentry where pid = 2930773953")
data = db.fetchall()

In [ ]:
#plt.scatter(*zip(*data));
p = pd.DataFrame(data, columns = ['a', 'e', 'i', 'ascendingNode', 'l', 'f' ])

In [ ]:
#p.plot.scatter('a','e');
#p.e.plot.line();
scatter_matrix(p, alpha=0.2, figsize=(8, 8), diagonal='kde');

In [ ]:
#non-resonant pid => 2930773953
#resonant pid => 87897423
db.execute("select a, e, i, ascendingNode, l, f, resonantAngle from particlesimentry where pid = 87897423")
data = db.fetchall()

In [ ]:
#plt.scatter(*zip(*data));
p = pd.DataFrame(data, columns = ['a', 'e', 'i', 'ascendingNode', 'l', 'f', 'resonantAngle' ])

In [ ]:
#p.plot.scatter('a','e');
#p.plot.line(y='e');
from pandas.plotting import scatter_matrix
scatter_matrix(p, alpha=0.2, figsize=(8, 8), diagonal='kde');

In [ ]:
p.resonantAngle.plot.line();

In [ ]:
db.execute("select pid, AVG(resonantAngle), MAX(resonantAngle) as resmax, MIN(resonantAngle) as resmin FROM particlesimentry p "
        ' WHERE pid NOT IN (SELECT pid FROM "DiscardedParticles") '
        " GROUP BY pid "
#        " HAVING MAX(resonantAngle) - MIN(resonantAngle) <= 175"
        " HAVING MAX(resonantAngle) - MIN(resonantAngle) <= 350"
              )
data = db.fetchall()

In [ ]:
len(data)

In [ ]:
d = [x[1] for x in data]

binwidth = 2
bins = np.arange(min(d), max(d) + binwidth, binwidth)


plt.rcParams['figure.figsize'] = (15, 5)
plt.hist(d, bins=bins);

In [ ]:
d = [x[2] - x[3] for x in data]

binwidth = 20
bins = np.arange(min(d), max(d) + binwidth, binwidth)


plt.rcParams['figure.figsize'] = (8, 5)
plt.hist(d, bins=bins);
plt.xlabel("amplitude");
plt.ylabel("number of particles");


In [ ]:
frame = pd.DataFrame(d)
frame.plot.kde();

In [ ]:
resonantCandidates = [x for x in data if x[1] >= 175 and x[1] <= 185]
len(resonantCandidates)

In [ ]:
db.execute("select a, e FROM particlesimentry AS p"
        " WHERE t = 10000000 AND p.pid IN ( " + ','.join([str(x[0]) for x in resonantCandidates]) + ")"
          )

lastParameters = db.fetchall()



In [ ]:
eccentricities = [x[1] for x in lastParameters]

binwidth = 0.05
bins = np.arange(min(eccentricities), max(eccentricities) + binwidth, binwidth)


plt.rcParams['figure.figsize'] = (8, 5)
plt.hist(eccentricities, bins=bins);
plt.xlabel("eccentricity");
plt.ylabel("number of particles");


In [ ]:
frame = pd.DataFrame(eccentricities)
frame.plot.kde();

In [ ]:
db.execute("select p.a / neptune.a, p.e FROM particlesimentry AS p"
        " JOIN planetsimentry as neptune ON neptune.planet = 3 AND neptune.t = p.t "
        " WHERE p.pid IN ( " + ','.join([str(x[0]) for x in resonantCandidates]) + ") "
          )

resonantParameters = db.fetchall()

In [ ]:
plt.scatter(*zip(*resonantParameters), s = 0.1);
plt.xlim(1.805, 1.880);
plt.ylim(0, 1);
plt.rcParams['figure.figsize'] = (10, 8);
plt.xlabel("a/aN");
plt.ylabel("eccentricity");



In [ ]:
conn.close()